In [134]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
E = OneHotEncoder()
rng =np.random.default_rng()
np.set_printoptions(suppress=True,precision=4)


In [135]:
N=100
k=5
r=3
M=rng.uniform(size=(k,r))

In [136]:
def sigma(x,m):
    y = x @ m 
    j = np.exp(y)
    p = j/j.sum(axis=1,keepdims=True)
    return p

In [140]:
def simulate(M,n=100,k=5,r=3):
    x=rng.choice(list(range(10)),size=(n,k))
    P=sigma(x,M)
    Y=rng.multinomial(1,P)
    labels = np.argmax(Y,axis=1)
    return x, labels, P

In [142]:
x,y,P=simulate(M)
L=LogisticRegression()
L.fit(x,y)
P0=L.predict_proba(x)
P[:10,:],P0[:10,:]



(array([[0.8408, 0.0265, 0.1327],
        [0.1016, 0.0285, 0.8699],
        [0.971 , 0.0203, 0.0087],
        [0.8533, 0.1246, 0.0221],
        [0.0113, 0.0267, 0.962 ],
        [0.1143, 0.0122, 0.8735],
        [0.8551, 0.0167, 0.1282],
        [0.9479, 0.0163, 0.0358],
        [0.805 , 0.0202, 0.1748],
        [0.1107, 0.0329, 0.8564]]),
 array([[0.8873, 0.0054, 0.1073],
        [0.0478, 0.0032, 0.949 ],
        [0.9556, 0.0419, 0.0025],
        [0.9829, 0.0104, 0.0067],
        [0.015 , 0.0479, 0.9371],
        [0.062 , 0.0089, 0.9291],
        [0.7667, 0.0809, 0.1524],
        [0.9679, 0.0041, 0.0281],
        [0.7784, 0.0015, 0.2201],
        [0.0544, 0.0006, 0.945 ]]))

In [91]:
def descent(x,labels):
    N=x.shape[0]
    k=x.shape[1]
    Y=E.fit_transform(labels.reshape(-1,1)).toarray()
    M=rng.uniform(size=(k,r))
    P0 = sigma(x,M)
    for i in range(100):
        
    

In [92]:
Y=rng.multinomial(1,P)

In [93]:
P[:10,:], Y[:10,:]

(array([[0.2356, 0.0747, 0.6896],
        [0.2909, 0.4769, 0.2322],
        [0.0016, 0.0992, 0.8992],
        [0.0217, 0.1373, 0.8409],
        [0.2019, 0.0969, 0.7012],
        [0.0073, 0.8651, 0.1277],
        [0.2399, 0.4472, 0.3129],
        [0.0307, 0.6544, 0.3149],
        [0.0042, 0.8968, 0.099 ],
        [0.2903, 0.008 , 0.7017]]),
 array([[0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1]]))

In [94]:
Y.shape

(500, 3)

In [95]:
x

array([[0, 6, 3, 0, 6],
       [1, 0, 3, 3, 6],
       [6, 7, 2, 8, 4],
       ...,
       [3, 7, 6, 6, 2],
       [9, 2, 4, 1, 9],
       [1, 6, 0, 0, 6]])

In [96]:
from sklearn.linear_model import LogisticRegression

In [97]:
labels=np.argmax(Y,axis=1)

In [98]:
L=LogisticRegression()
L.fit(x,labels)

LogisticRegression()

In [99]:
L.coef_

array([[ 0.0645, -0.1242,  0.0534, -0.3603,  0.0504],
       [-0.2543, -0.1098, -0.203 ,  0.3752,  0.0337],
       [ 0.1898,  0.234 ,  0.1495, -0.0149, -0.0842]])

In [100]:
M

array([[0.3128, 0.1606, 0.5378],
       [0.4795, 0.5343, 0.919 ],
       [0.827 , 0.4631, 0.7968],
       [0.2816, 0.9893, 0.6525],
       [0.9121, 0.848 , 0.6667]])

In [101]:
L.intercept_

array([ 0.307 , -0.1493, -0.1577])

In [102]:
labels

array([2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 1,
       0, 0, 2, 2, 2, 2, 1, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2,
       2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2,
       2, 1, 1, 1, 2, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 1, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2,
       1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 0, 1, 1,

In [103]:
L.predict_proba(x)

array([[0.2224, 0.0644, 0.7132],
       [0.2532, 0.4442, 0.3025],
       [0.0049, 0.1014, 0.8937],
       ...,
       [0.0079, 0.0363, 0.9558],
       [0.2874, 0.0068, 0.7058],
       [0.2393, 0.1087, 0.652 ]])

In [104]:
P

array([[0.2356, 0.0747, 0.6896],
       [0.2909, 0.4769, 0.2322],
       [0.0016, 0.0992, 0.8992],
       ...,
       [0.0048, 0.0308, 0.9644],
       [0.274 , 0.0207, 0.7053],
       [0.1717, 0.1394, 0.689 ]])

In [115]:
((np.argmax(L.predict_proba(x),axis=1)-np.argmax(P,axis=1))!=0).sum()

24

In [112]:
np.argmax(P,axis=1)

array([2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 1, 1, 2, 1, 2, 2, 2, 0, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 0, 2, 2, 1, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 1, 2, 2, 1, 2,
       2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2,
       1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 1, 2, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2,

In [109]:
P.shape

(500, 3)

In [110]:
x.shape

(500, 5)

In [117]:
Y[:20,:]

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0]])

In [118]:
(Y-P)

array([[-0.2356, -0.0747,  0.3104],
       [-0.2909, -0.4769,  0.7678],
       [-0.0016, -0.0992,  0.1008],
       ...,
       [-0.0048, -0.0308,  0.0356],
       [-0.274 , -0.0207,  0.2947],
       [-0.1717, -0.1394,  0.311 ]])

In [119]:
(Y-P).shape

(500, 3)

In [120]:
x.shape

(500, 5)

In [121]:
J=(Y-P)

In [127]:
(J[:,2].reshape(500,1)*x).sum(axis=0)

array([16.0576, -1.8316, 51.9075, 19.8464, 70.3034])

In [143]:
from sklearn.preprocessing import OneHotEncoder


In [145]:
E=OneHotEncoder()

In [151]:
Y=E.fit_transform(labels.reshape(-1,1)).toarray()

In [152]:
Y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [148]:
E.transform(labels).toarray()

ValueError: X does not contain any features, but OneHotEncoder is expecting 1 features